# RAM Concept Creep Factor <v0.1>

## Introduction:

This notebook aids in the calculation of the RAM Concept concrete creep factor. The RAM Concept creep factor defaults to a value of 3.35 based on recommendations in ACI 209.

### Assumptions and Limitations:
- Valid only for moist-cured slab elements with age at loading greater than 7 days.

## Project Information:

- **Company**: Structura
- **Project No. and Name**: 
- **Date**: 
- **Prepared By**:
- **Checked By**:

## Inputs:
- $t_0$ [days] = age of slab at application of loading. E.g. $t_0 = 14$ for 14 days.
- $h$ [in] = slab depth.
- $s$ [in] = concrete mix slump.
- $psi$ [%] = $\psi$, ratio of fine aggregate to total aggregate (typically about 40%, i.e. $\psi = 40$).
- $\alpha$ [%] = $\alpha$, air content. E.g. $\alpha = 1.5$ for 1.5% air content.
- $RH$ = average relative humidity. E.g. $RH = 0.7$ for 70%.

Expected units are given with square brackets, '[  ]'.

In [3]:
import ipywidgets as widgets
from IPython.display import display
import json

class Creep_Factor(object):
    
    def __init__(self):
        
        self.t0 = 14
        self.h = 8
        self.s = 3.5
        self.p = 40
        self.a = 1.5
        self.RH = 0.7

        self.t0_wgt = widgets.FloatText(value=self.t0, description=r'$t_0$ &nbsp <i>[days]</i>', disabled=False)
        self.h_wgt = widgets.FloatText(value=self.h, description=r'$h$ &nbsp <i>[in]</i>', disabled=False)
        self.s_wgt = widgets.FloatText(value=self.s, description=r'$s$ &nbsp <i>[in]</i>', disabled=False)
        self.p_wgt = widgets.FloatText(value=self.p, description=r'$\psi$ &nbsp <i>[%]</i>', disabled=False)
        self.a_wgt = widgets.FloatText(value=self.a, description=r'$\alpha$ &nbsp <i>[%]</i>', disabled=False)
        self.RH_wgt = widgets.FloatText(value=self.RH, description=r'$RH$ &nbsp <i>[u]</i>', disabled=False)
        
        self.calc_btn = widgets.Button(description='Calc')
        self.calc_btn.on_click(self.recalc)
        
        self.save_btn = widgets.Button(description='Save')
        self.save_btn.on_click(self.save)
        
        self.output_wgt = widgets.Output()
        with self.output_wgt:
            print('needs calc')
                
        self.data = {
            'input': {
                't0': self.t0,
                'h': self.h,
                's': self.s,
                'p': self.p,
                'a': self.a,
                'RH': self.RH},
            'output': {
                'gamma_ct0': 'needs calc',
                'gamma_ch': 'needs calc',
                'gamma_cs': 'needs calc',
                'gamma_cpsi': 'needs calc',
                'gamma_calpha': 'needs calc',
                'gamma_cRH': 'needs calc',
                'gamma_c': 'needs calc',
                'phi_u': 'needs calc',
                'k_e': 'needs calc'}}

    def recalc(self, b):
        
        self.output_wgt.clear_output()

        self.t0 = self.t0_wgt.value
        self.h = self.h_wgt.value
        self.s = self.s_wgt.value
        self.p = self.p_wgt.value
        self.a = self.a_wgt.value
        self.RH = self.RH_wgt.value

        # Calculate the concrete creep factor.
        self.gamma_ct0 = 1.25*self.t0**-0.118
        self.gamma_ch = 1.10 - 0.017*self.h
        self.gamma_cs = 0.82 + 0.067*self.s
        self.gamma_cpsi = 0.88 + 0.0024*self.p
        self.gamma_calpha = max(1, 0.46 + 0.09*self.a)
        self.gamma_cRH = 1.27 - 0.67*self.RH
        self.gamma_c = self.gamma_ct0*self.gamma_ch*self.gamma_cs*self.gamma_cpsi*self.gamma_calpha*self.gamma_cRH
        self.phi_u = 2.35*self.gamma_c
        self.k_e = self.phi_u + 1

        # Save results.
        self.data['input']['t0'] = self.t0
        self.data['input']['h'] = self.h
        self.data['input']['s'] = self.s
        self.data['input']['p'] = self.p
        self.data['input']['a'] = self.a
        self.data['input']['RH'] = self.RH
        self.data['output']['gamma_ct0'] = self.gamma_ct0
        self.data['output']['gamma_ch'] = self.gamma_ch
        self.data['output']['gamma_cs'] = self.gamma_cs
        self.data['output']['gamma_cpsi'] = self.gamma_cpsi
        self.data['output']['gamma_calpha'] = self.gamma_calpha
        self.data['output']['gamma_cRH'] = self.gamma_cRH
        self.data['output']['gamma_c'] = self.gamma_c
        self.data['output']['phi_u'] = self.phi_u
        self.data['output']['k_e'] = self.k_e
            
        with self.output_wgt:
            display(widgets.HTMLMath(
                rf"""
                <h3> User Input </h3>
                $t_0 = \boldsymbol{{ {self.t0} }}$ days &nbsp; <i>(age at loading)</i> <br>
                $h = \boldsymbol{{ {self.h} }}$ in &nbsp; <i>(slab thickness)</i> <br>
                $s = \boldsymbol{{ {self.s} }}$ in &nbsp; <i>(slump)</i> <br>
                $\psi = \boldsymbol{{ {self.p} }}$ % &nbsp; <i>(fine aggregate to toal aggregate)</i> <br>
                $\alpha = \boldsymbol{{ {self.a} }}$ % &nbsp; <i>(air content)</i> <br>
                $RH = \boldsymbol{{ {self.RH} }}$ &nbsp; <i>(relative humidity)</i> <br>

                <h3> Calculate Creep Factor </h3>
                $\gamma_{{c,t0}} = 1.25t_0^{{-0.118}} = \boldsymbol{{ {self.gamma_ct0:.2f} }}$ &nbsp; <i>(age at loading factor)</i> <br>
                $\gamma_{{c,h}} = 1.10 - 0.017h = \boldsymbol{{ {self.gamma_ch:.2f} }}$ &nbsp; <i>(thickness factor)</i> <br>
                $\gamma_{{c,s}} = 0.82 + 0.067s = \boldsymbol{{ {self.gamma_cs:.2f} }}$ &nbsp; <i>(slump factor)</i> <br>
                $\gamma_{{c,\psi}} = 0.88 + 0.0024\psi = \boldsymbol{{ {self.gamma_cpsi:.2f} }}$ &nbsp; <i>(fine aggregate to total aggregate factor)</i> <br>
                $\gamma_{{c,\alpha}} = max(1, 0.46 + 0.09\alpha) = \boldsymbol{{ {self.gamma_calpha:.2f} }}$ &nbsp; <i>(air content factor)</i> <br>
                $\gamma_{{c,RH}} = 1.27 - 0.67RH = \boldsymbol{{ {self.gamma_cRH:.2f} }}$ &nbsp; <i>(relative humidity factor)</i> <br>
                $\gamma_c = \gamma_{{c,t0}}\gamma_{{c,RH}}\gamma_{{c,d}}\gamma_{{c,s}}\gamma_{{c,\psi}}\gamma_{{c,\phi}}\gamma_{{c,\alpha}} = \boldsymbol{{ {self.gamma_c:.2f} }}$ &nbsp; <i>(correction factor)</i>  <br>
                    <blockquote>
                    $\phi_u = 2.35\gamma_c = \boldsymbol{{ {self.phi_u:.2f} }}$ &nbsp; <i>(creep coefficient)</i> <br>
                        <blockquote>
                        $k_{{e}} = \phi_{{u}} + 1 = \boldsymbol{{ {self.k_e:.2f} }}$ &nbsp; <i>(Concept creep factor)</i>
                        </blockquote>
                    </blockquote>
                """))

    def save(self, b):

        self.data = {
            'input': {
                't0': self.t0,
                'h': self.h,
                's': self.s,
                'p': self.p,
                'a': self.a,
                'RH': self.RH},
            'output': {
                'gamma_ct0': self.gamma_ct0,
                'gamma_ch': self.gamma_ch,
                'gamma_cs': self.gamma_cs,
                'gamma_cpsi': self.gamma_cpsi,
                'gamma_calpha': self.gamma_calpha,
                'gamma_cRH': self.gamma_cRH,
                'gamma_c': self.gamma_c,
                'phi_u': self.phi_u,
                'k_e': self.k_e}}

        with open('data.txt', 'w') as outfile:
            json.dump(self.data, outfile)
                
cf = Creep_Factor()
display(cf.t0_wgt, cf.h_wgt, cf.s_wgt, cf.p_wgt, cf.a_wgt, cf.RH_wgt)
display(cf.calc_btn)

FloatText(value=14.0, description='$t_0$ &nbsp <i>[days]</i>')

FloatText(value=8.0, description='$h$ &nbsp <i>[in]</i>')

FloatText(value=3.5, description='$s$ &nbsp <i>[in]</i>')

FloatText(value=40.0, description='$\\psi$ &nbsp <i>[%]</i>')

FloatText(value=1.5, description='$\\alpha$ &nbsp <i>[%]</i>')

FloatText(value=0.7, description='$RH$ &nbsp <i>[u]</i>')

Button(description='Calc', style=ButtonStyle())

## Results:

In [2]:
display(cf.output_wgt)

Output()

Button(description='Save', style=ButtonStyle())

## Save/Load:

In [ ]:
display(cf.save_btn)

## Discussion:
As noted in the RAM Concept manual, the creep coefficient ($\phi_u$) is commonly defined as follows. 
> $\phi_u = \frac{\epsilon_c}{\epsilon_i}$, where
>> $\epsilon_c =$ creep strain, and <br>
>> $\epsilon_i = \epsilon_e =$ initial (elastic) strain

> $\epsilon_f = \epsilon_i + \epsilon_c$, where
>> $\epsilon_f =$ total strain

RAM Concept uses the factor $k_e$ to symbolize the "concrete design creep factor" and defined it as follows.
> $k_e = \frac{\epsilon_f}{\epsilon_e}$, where $\epsilon_e = \epsilon_i$

Consequently, Concept uses 
> $k_e = \frac{\epsilon_i + \epsilon_c}{\epsilon_i} = 1 + \frac{\epsilon_c}{\epsilon_i} = 1 + \phi_u$

Since 2.35 is suggested as average creep coefficient, $\phi_u = 2.35$, Concept defaults to $k_e = 3.35$.

Average relative humidity for different regions in the U.S. may be found the PCI Design Handbook.

### References:

1. ACI 209.2R-08 - Guide for Modeling and Calculating Shrinkage and Creep in Hardened Concrete, ACI, 2008.
2. RAM Concept Manual, 2020.
3. Prestressed Concrete Analysis and Design - Fundamentals (3rd Ed.), Antoine E. Naaman, 2012.

### Change Log:
- Created: Dec. 21, 2020 by SJ
- Released:

### Comments:
- Where can each of the inputs be referenced? 
- How much difference does each factor make?
- Can we get a chart of creep factors for common situations and for reference (controls)?